In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import tensorflow
import os
import cv2
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import shutil
import soundfile as sf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from skimage.transform import resize
from keras.applications.resnet50 import preprocess_input

# Preprocess data

In [ ]:
dataset_path = '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_original'
split_path = '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_split'

mel_spectrogram_path = '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms_mel'
mfcc_spectrogram_path = '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms_mfcc'
standard_spectrogram_path = '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms_standard'

one_second_path = '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_1s'
one_second_spectrogram_path ='/content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms_1s'

three_second_path = '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_3s'
three_second_spectrogram_path ='/content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms_3s'

five_second_path = '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_5s'
five_second_spectrogram_path ='/content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms_5s'

ten_second_path = '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_10s'
ten_second_spectrogram_path ='/content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms_10s'

twenty_second_path = '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s'
twenty_spectrogram_path ='/content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms_20s'

## Split data into test and train subsets
This will allow track segmentation without data interference

### Split audio data

In [ ]:
if not os.path.exists(split_path):
    os.makedirs(split_path)

genres = os.listdir(dataset_path)

for genre in genres:
  genre_path = os.path.join(dataset_path, genre)

  genre_output_path = os.path.join(split_path, genre)
  os.makedirs(genre_output_path, exist_ok=True)

  tracks = [track for track in os.listdir(genre_path) if track.endswith('.wav')]

  train_tracks, test_tracks = train_test_split(tracks, test_size=0.2, random_state=42)

  train_output_path = os.path.join(genre_output_path, "train")
  os.makedirs(train_output_path, exist_ok=True)
  for track in train_tracks:
      shutil.copy(os.path.join(genre_path, track), os.path.join(train_output_path, track))

  test_output_path = os.path.join(genre_output_path, "test")
  os.makedirs(test_output_path, exist_ok=True)
  for track in test_tracks:
      shutil.copy(os.path.join(genre_path, track), os.path.join(test_output_path, track))


### Split image data

In [ ]:
def split_original_image_data(input_path, output_path, random_state):
  if not os.path.exists(output_path):
      os.makedirs(output_path)

  train_output_path = os.path.join(output_path, "train")
  test_output_path = os.path.join(output_path, "test")
  os.makedirs(train_output_path, exist_ok=True)
  os.makedirs(test_output_path, exist_ok=True)

  genres = os.listdir(input_path)

  for genre in genres:
    genre_path = os.path.join(input_path, genre)

    train_genre_output_path = os.path.join(train_output_path, genre)
    test_genre_output_path = os.path.join(test_output_path, genre)
    os.makedirs(train_genre_output_path, exist_ok=True)
    os.makedirs(test_genre_output_path, exist_ok=True)

    tracks = os.listdir(genre_path)

    train_tracks, test_tracks = train_test_split(tracks, test_size=0.2, random_state=random_state)

    for track in train_tracks:
        shutil.copy(os.path.join(genre_path, track), os.path.join(train_genre_output_path, track))

    for track in test_tracks:
        shutil.copy(os.path.join(genre_path, track), os.path.join(test_genre_output_path, track))


In [ ]:
def split_splitted_image_data(structure_path, input_path, output_path):
  if not os.path.exists(structure_path):
      os.makedirs(structure_path)
  if not os.path.exists(output_path):
      os.makedirs(output_path)

  train_output_path = os.path.join(output_path, "train")
  test_output_path = os.path.join(output_path, "test")
  os.makedirs(train_output_path, exist_ok=True)
  os.makedirs(test_output_path, exist_ok=True)

  split_subtypes = os.listdir(structure_path)

  for split_subtype in split_subtypes:
    split_subtype_path = os.path.join(structure_path, split_subtype)

    genres = os.listdir(split_subtype_path)

    for genre in genres:
      genre_structure_path = os.path.join(split_subtype_path, genre)
      genres_input_path = os.path.join(input_path, genre)

      genres_train_output_path = os.path.join(train_output_path, genre)
      genres_test_output_path = os.path.join(test_output_path, genre)
      os.makedirs(genres_train_output_path, exist_ok=True)
      os.makedirs(genres_test_output_path, exist_ok=True)

      structure_tracks = os.listdir(genre_structure_path)
      input_tracks = os.listdir(genres_input_path)

      for input_track in input_tracks:
        clip_base_name = os.path.splitext(input_track)[0]

        for structure_track in structure_tracks:
          full_track_base_name = os.path.splitext(structure_track)[0]

          if full_track_base_name in clip_base_name:
            if split_subtype == 'train':
              shutil.copy(os.path.join(genres_input_path, input_track), os.path.join(genres_train_output_path, input_track))
            else:
              shutil.copy(os.path.join(genres_input_path, input_track), os.path.join(genres_test_output_path, input_track))


In [ ]:
def split_zip_and_copy(split_type, random_state, input_path, output_path):
  if split_type == 'original':
    split_original_image_data(f'/content/spectrograms_{split_type}_not_splitted/',
                              f'/content/spectrograms_{split_type}_randomstate_{random_state}', random_state)
  else:
    split_splitted_image_data('/content/spectrograms_original_randomstate_96/', input_path, output_path)

  folder_name = f'spectrograms_{split_type}_randomstate_96'
  split_name = f'spectrograms_{split_type}'
  !zip -q -r /content/{split_name}.zip /content/{folder_name}/
  !cp /content/{split_name}.zip /content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms/Spectrograms_with_test_train_split/Split_3_randomstate_96/

In [ ]:
split_zip_and_copy('split_20s', 96, '/content/spectrograms_split_20s_not_splitted/', '/content/spectrograms_split_20s_randomstate_96/')

In [ ]:
!zip -q -r spectrograms_original.zip /content/spectrograms_original_randomstate_96/

In [ ]:
!cp spectrograms_original.zip /content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms/Spectrograms_with_test_train_split/

## Resplit data from test and train subsets

In [ ]:
def merge_from_split_subsets(input_path, output_path):
  if not os.path.exists(output_path):
    os.makedirs(output_path)

  for split_subset in os.listdir(input_path):
    split_subset_path = os.path.join(input_path, split_subset)
    for genre in os.listdir(split_subset_path):
        genre_path = os.path.join(split_subset_path, genre)
        output_genre_path = os.path.join(output_path, genre)
        os.makedirs(output_genre_path, exist_ok=True)
        for track in os.listdir(genre_path):
            track_path = os.path.join(genre_path, track)
            shutil.copy(track_path, output_genre_path)

In [ ]:
!unzip '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms/Spectrograms_with_test_train_split/spectrograms_mel.zip' -d "/content/spectrograms_original/"

merge_from_split_subsets('/content/spectrograms_original/', '/content/spectrograms_original_not_splitted')

Archive:  /content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms/Spectrograms_with_test_train_split/spectrograms_mel.zip
   creating: /content/spectrograms_original/test/
   creating: /content/spectrograms_original/test/blues/
  inflating: /content/spectrograms_original/test/blues/blues.00006.png  
  inflating: /content/spectrograms_original/test/blues/blues.00007.png  
  inflating: /content/spectrograms_original/test/blues/blues.00010.png  
  inflating: /content/spectrograms_original/test/blues/blues.00011.png  
  inflating: /content/spectrograms_original/test/blues/blues.00018.png  
  inflating: /content/spectrograms_original/test/blues/blues.00022.png  
  inflating: /content/spectrograms_original/test/blues/blues.00023.png  
  inflating: /content/spectrograms_original/test/blues/blues.00031.png  
  inflating: /content/spectrograms_original/test/blues/blues.00032.png  
  inflating: /content/spectrograms_original/test/blues/blues.00036.png  
  inflating: /content/spectrograms_ori

In [ ]:
!zip -r /content/spectrograms_original_not_splitted.zip /content/spectrograms_original_not_splitted/

  adding: content/spectrograms_original_not_splitted/ (stored 0%)
  adding: content/spectrograms_original_not_splitted/disco/ (stored 0%)
  adding: content/spectrograms_original_not_splitted/disco/disco.00062.png (deflated 14%)
  adding: content/spectrograms_original_not_splitted/disco/disco.00006.png (deflated 7%)
  adding: content/spectrograms_original_not_splitted/disco/disco.00032.png (deflated 7%)
  adding: content/spectrograms_original_not_splitted/disco/disco.00081.png (deflated 14%)
  adding: content/spectrograms_original_not_splitted/disco/disco.00002.png (deflated 8%)
  adding: content/spectrograms_original_not_splitted/disco/disco.00014.png (deflated 4%)
  adding: content/spectrograms_original_not_splitted/disco/disco.00076.png (deflated 6%)
  adding: content/spectrograms_original_not_splitted/disco/disco.00023.png (deflated 7%)
  adding: content/spectrograms_original_not_splitted/disco/disco.00044.png (deflated 8%)
  adding: content/spectrograms_original_not_splitted/disco/

In [ ]:
!cp /content/spectrograms_original_not_splitted.zip /content/drive/MyDrive/GoogleCollab/Data/GTZAN/

## Track segmentation

In [ ]:
def split_audio(input_path, output_path, split_length_seconds):
  if not os.path.exists(output_path):
          os.makedirs(output_path)

  for genre_dir in os.listdir(input_path):
      genre_dir_path = os.path.join(input_path, genre_dir)
      if os.path.isdir(genre_dir_path):
          output_genre_dir_path = os.path.join(output_path, genre_dir)
          if not os.path.exists(output_genre_dir_path):
              os.makedirs(output_genre_dir_path)

          # Loop through subdirectories
          for split_dir in os.listdir(genre_dir_path):
              split_dir_path = os.path.join(genre_dir_path, split_dir)
              if os.path.isdir(split_dir_path) and split_dir in ['train', 'test']:
                  output_split_dir_path = os.path.join(output_genre_dir_path, split_dir)
                  if not os.path.exists(output_split_dir_path):
                      os.makedirs(output_split_dir_path)

              # Loop through audio files
              for file in os.listdir(split_dir_path):
                  if file.endswith('.wav') and file != 'classical.00087.wav':
                      audio_path = os.path.join(split_dir_path, file)
                      y, sr = sf.read(audio_path)

                      # Calculate number of samples for x second
                      clip_length = int(sr * split_length_seconds)

                      # Split audio into x-second clips
                      for i in range(len(y) // clip_length):
                          clip = y[i*clip_length : (i+1)*clip_length]
                          output_filename = os.path.splitext(file)[0] + f'_clip{i}.wav'
                          output_file_path = os.path.join(output_split_dir_path, output_filename)
                          sf.write(output_file_path, clip, sr)
                          print(f"Saved {output_file_path}")

In [ ]:
def strip_to_20s(input_path, output_path):
  if not os.path.exists(output_path):
          os.makedirs(output_path)

  for genre_dir in os.listdir(input_path):
      genre_dir_path = os.path.join(input_path, genre_dir)
      if os.path.isdir(genre_dir_path):
          output_genre_dir_path = os.path.join(output_path, genre_dir)
          if not os.path.exists(output_genre_dir_path):
              os.makedirs(output_genre_dir_path)

          # Loop through subdirectories
          for split_dir in os.listdir(genre_dir_path):
              split_dir_path = os.path.join(genre_dir_path, split_dir)
              if os.path.isdir(split_dir_path) and split_dir in ['train', 'test']:
                  output_split_dir_path = os.path.join(output_genre_dir_path, split_dir)
                  if not os.path.exists(output_split_dir_path):
                      os.makedirs(output_split_dir_path)

              # Loop through audio files
              for file in os.listdir(split_dir_path):
                if file.endswith('.wav') and file != 'classical.00087.wav':
                    audio_path = os.path.join(split_dir_path, file)
                    y, sr = sf.read(audio_path)

                    # Calculate number of samples for the strip length
                    strip_length = int(sr * 20)

                    # Get the last strip_length samples
                    if len(y) > strip_length:
                        y = y[-strip_length:]

                    output_file_path = os.path.join(output_split_dir_path, file)
                    sf.write(output_file_path, y, sr)
                    print(f"Saved {output_file_path}")


In [ ]:
split_audio(split_path, ten_second_path, 10)

Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_10s/reggae/train/reggae.00043_clip0.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_10s/reggae/train/reggae.00043_clip1.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_10s/reggae/train/reggae.00043_clip2.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_10s/reggae/train/reggae.00091_clip0.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_10s/reggae/train/reggae.00091_clip1.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_10s/reggae/train/reggae.00091_clip2.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_10s/reggae/train/reggae.00021_clip0.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_10s/reggae/train/reggae.00021_clip1.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_10s/reggae/train/reggae.00021_clip2.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_10s/reggae/train/reggae.00034_clip0.wav


In [ ]:
strip_to_20s(split_path, twenty_second_path)

Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00043.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00037.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00034.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00021.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00091.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00009.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00097.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00079.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00008.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00076.wav
Saved /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_

In [ ]:
!zip -r /content/genres_20s.zip /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s

  adding: content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/ (stored 0%)
  adding: content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/ (stored 0%)
  adding: content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/ (stored 0%)
  adding: content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00043.wav (deflated 6%)
  adding: content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00037.wav (deflated 11%)
  adding: content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00034.wav (deflated 10%)
  adding: content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00021.wav (deflated 5%)
  adding: content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00091.wav (deflated 9%)
  adding: content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/reggae/train/reggae.00009.wav (deflated 9%)
  adding: content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_20s/regga

In [ ]:
from google.colab import files
files.download("/content/genres_20s.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Generate MEL spectrograms

In [ ]:
def generate_spectrogram(file_path, save_path, spectrogram_type):
    HOP_LENGTH = 512
    N_FFT = 512
    N_MELS = 128
    y, sr = librosa.load(file_path)

    if spectrogram_type == 'standard':
      spectrogram = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)
    elif spectrogram_type == 'mel':
      mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, hop_length=HOP_LENGTH, n_mels=N_MELS)
      spectrogram = librosa.amplitude_to_db(mel_spec, ref=np.max)
    elif spectrogram_type == 'mfcc':
        spectrogram = librosa.feature.mfcc(y=y, sr=sr)

    plt.figure(figsize=(6, 3))
    librosa.display.specshow(spectrogram, x_axis='time', y_axis='mel', sr=sr, hop_length=HOP_LENGTH)
    plt.axis('off')
    temp_save_path = "temp_spectrogram.png"
    plt.savefig(temp_save_path)
    plt.close()

    return cv2.imwrite(save_path, cv2.imread(temp_save_path))

## Create spectrogram subdirectories based on audio files directory structure

In [ ]:
def create_directories(audio_path, spectrogram_path):
    os.makedirs(spectrogram_path, exist_ok=True)

    for genre_dir in os.listdir(audio_path):
        genre_dir_path = os.path.join(audio_path, genre_dir)
        if os.path.isdir(genre_dir_path):
            spectrogram_genre_dir_path = os.path.join(spectrogram_path, genre_dir)
            if not os.path.exists(spectrogram_genre_dir_path):
                os.makedirs(spectrogram_genre_dir_path)

            for split_dir in os.listdir(genre_dir_path):
                split_dir_path = os.path.join(genre_dir_path, split_dir)
                if os.path.isdir(split_dir_path) and split_dir in ['train', 'test']:
                    spectrogram_split_dir_path = os.path.join(spectrogram_genre_dir_path, split_dir)
                    if not os.path.exists(spectrogram_split_dir_path):
                        os.makedirs(spectrogram_split_dir_path)

## Save spectrograms as .png

In [ ]:
import os
import shutil

def save_spectrogram(audio_path, spectrogram_path, spectrogram_type):
    for genre in os.listdir(audio_path):
      genre_dir_path = os.path.join(audio_path, genre)
      print(f"Starting generation of genre: {genre}")
      for split_subset in os.listdir(genre_dir_path):
          split_dir_path = os.path.join(genre_dir_path, split_subset)

          for filename in os.listdir(split_dir_path):
            if filename != 'classical.00087.wav':
              file_path = os.path.join(split_dir_path, filename)
              save_dir_fragment = split_dir_path.split(audio_path.split('/content/', 1)[1], 1)
              stripped_path = save_dir_fragment[1].lstrip('/')

              save_file_path = os.path.join(spectrogram_path, stripped_path, f"{filename.replace('.wav', '.png')}")
              generate_spectrogram(file_path, save_file_path, spectrogram_type)
          print(f"Done generating: genre: {genre}, subset: {split_subset}, track count: {len(os.listdir(split_dir_path))}")

In [ ]:
# Processing thorugh local storage is way faster than GDrive
!unzip '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_split.zip' -d "/content/"

unzip:  cannot find or open /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_split.zip, /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_split.zip.zip or /content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_split.zip.ZIP.


In [ ]:
create_directories('/content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_split', '/content/spectrograms_mfcc')

In [ ]:
save_spectrogram('/content/drive/MyDrive/GoogleCollab/Data/GTZAN/genres_split', '/content/spectrograms_mfcc', 'mfcc')

Starting generation of genre: reggae
Done generating: genre: reggae, subset: train, track count: 80
Done generating: genre: reggae, subset: test, track count: 20
Starting generation of genre: blues
Done generating: genre: blues, subset: train, track count: 80
Done generating: genre: blues, subset: test, track count: 20
Starting generation of genre: metal
Done generating: genre: metal, subset: train, track count: 80
Done generating: genre: metal, subset: test, track count: 20
Starting generation of genre: country
Done generating: genre: country, subset: train, track count: 80
Done generating: genre: country, subset: test, track count: 20
Starting generation of genre: rock
Done generating: genre: rock, subset: train, track count: 80
Done generating: genre: rock, subset: test, track count: 20
Starting generation of genre: hiphop
Done generating: genre: hiphop, subset: train, track count: 80
Done generating: genre: hiphop, subset: test, track count: 20
Starting generation of genre: classic

## Zip and download due to Colab speed restrictions

In [ ]:
!zip -r /content/spectrograms_mfcc.zip /content/spectrograms_mfcc/

  adding: content/spectrograms_mfcc/ (stored 0%)
  adding: content/spectrograms_mfcc/reggae/ (stored 0%)
  adding: content/spectrograms_mfcc/reggae/train/ (stored 0%)
  adding: content/spectrograms_mfcc/reggae/train/reggae.00031.png (deflated 32%)
  adding: content/spectrograms_mfcc/reggae/train/reggae.00094.png (deflated 36%)
  adding: content/spectrograms_mfcc/reggae/train/reggae.00040.png (deflated 42%)
  adding: content/spectrograms_mfcc/reggae/train/reggae.00077.png (deflated 48%)
  adding: content/spectrograms_mfcc/reggae/train/reggae.00033.png (deflated 39%)
  adding: content/spectrograms_mfcc/reggae/train/reggae.00048.png (deflated 37%)
  adding: content/spectrograms_mfcc/reggae/train/reggae.00074.png (deflated 31%)
  adding: content/spectrograms_mfcc/reggae/train/reggae.00036.png (deflated 34%)
  adding: content/spectrograms_mfcc/reggae/train/reggae.00009.png (deflated 38%)
  adding: content/spectrograms_mfcc/reggae/train/reggae.00059.png (deflated 37%)
  adding: content/spect

In [ ]:
from google.colab import files
files.download("/content/spectrograms_mfcc.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Reshape directory trees and download

In [ ]:
def copy_files_to_new_directories(main_directory, new_test_dir, new_train_dir):
    # Traverse through the genre directories
    for genre_dir in os.listdir(main_directory):
        genre_path = os.path.join(main_directory, genre_dir)

        # Check if the genre directory contains 'test' and 'train' subdirectories
        if os.path.isdir(genre_path):
            test_path = os.path.join(genre_path, 'test')
            train_path = os.path.join(genre_path, 'train')

            new_test_genre_dir = os.path.join(new_test_dir, genre_dir)
            new_train_genre_dir = os.path.join(new_train_dir, genre_dir)

            # Create genre subdirectories in the new test and train directories if they don't exist
            if not os.path.exists(new_test_genre_dir):
                os.makedirs(new_test_genre_dir)
            if not os.path.exists(new_train_genre_dir):
                os.makedirs(new_train_genre_dir)

            if os.path.exists(test_path) and os.path.isdir(test_path):
                # Copy all files from current 'test' directory to new 'test' genre subdirectory
                for filename in os.listdir(test_path):
                    file_path = os.path.join(test_path, filename)
                    if os.path.isfile(file_path):
                        shutil.copy2(file_path, os.path.join(new_test_genre_dir, filename))

            if os.path.exists(train_path) and os.path.isdir(train_path):
                # Copy all files from current 'train' directory to new 'train' genre subdirectory
                for filename in os.listdir(train_path):
                    file_path = os.path.join(train_path, filename)
                    if os.path.isfile(file_path):
                        shutil.copy2(file_path, os.path.join(new_train_genre_dir, filename))

In [ ]:
!unzip '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms_5s.zip' -d "/content"

unzip:  cannot find or open /content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms_5s.zip, /content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms_5s.zip.zip or /content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms_5s.zip.ZIP.


In [ ]:
main_directory = '/content/spectrograms_mfcc'
new_test_dir = '/content/test'
new_train_dir = '/content/train'

copy_files_to_new_directories(main_directory, new_test_dir, new_train_dir)

In [ ]:
!zip -r /content/test.zip /content/test/
!zip -r /content/train.zip /content/train/

  adding: content/test/ (stored 0%)
  adding: content/test/reggae/ (stored 0%)
  adding: content/test/reggae/reggae.00045.png (deflated 35%)
  adding: content/test/reggae/reggae.00025.png (deflated 36%)
  adding: content/test/reggae/reggae.00051.png (deflated 38%)
  adding: content/test/reggae/reggae.00073.png (deflated 45%)
  adding: content/test/reggae/reggae.00015.png (deflated 32%)
  adding: content/test/reggae/reggae.00016.png (deflated 39%)
  adding: content/test/reggae/reggae.00004.png (deflated 43%)
  adding: content/test/reggae/reggae.00089.png (deflated 43%)
  adding: content/test/reggae/reggae.00075.png (deflated 33%)
  adding: content/test/reggae/reggae.00022.png (deflated 38%)
  adding: content/test/reggae/reggae.00039.png (deflated 34%)
  adding: content/test/reggae/reggae.00080.png (deflated 32%)
  adding: content/test/reggae/reggae.00071.png (deflated 54%)
  adding: content/test/reggae/reggae.00035.png (deflated 42%)
  adding: content/test/reggae/reggae.00002.png (defla

In [ ]:
from google.colab import files

files.download("/content/test.zip")
files.download("/content/train.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>